In [6]:
import numpy as np
import matplotlib.path as path
import import_ipynb
import utils

### Deadlock Function

`def deadlock(pos, bvc, parameters, robots_path, deadlock_pointers, verbose)`

Check if there are robots in deadlock situation

#### Parameters:
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Information of BVC
* **parameters&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Global parameters
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarryas of list of floats**
    * Previous positions
* **deadlock_pointers&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of ints**
    * Pointers to robots in deadlock
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * Flag for verbose
    
#### Return:
* **fixed_up_positions&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Positions to avoid deadlock

In [ ]:
def deadlock(pos, bvc, parameters, robots_path, deadlock_pointers, verbose,
             closer):
    # List to return
    fixed_up_positions = []
    # Check all robots positions
    for pointer in range(len(pos)):
        # Check if the robot is in deadlock situation
        if pointer in deadlock_pointers:
            # Initialize the instantaneous set of BVC vertices
            inst_vertices = bvc["vertices"][pointer]
            # Get the sense of the vertices_bvc array
            sense = utils.get_sense(inst_vertices, robots_path[-2][pointer])
            #i = 0
            milestone = -1
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Robot in deadlock:', pos[pointer])
                print('Vertices to check:', inst_vertices)
                print('Sense:', sense)
            main_distance = np.linalg.norm(closer["point"][pointer]
                                           - inst_vertices[0])
            for i in range(len(inst_vertices)):
                temp_distance = np.linalg.norm(closer["point"][pointer]
                                               - inst_vertices[i])
                if verbose:
                    print('Distance from vertex to the closest point:',
                          temp_distance)
                if (temp_distance < parameters["tolerance"]
                    and temp_distance < main_distance):
                    milestone = i
                    main_distance = temp_distance
                j = 0
                for j in range(parameters["previous"]):
                    prev_dist = np.linalg.norm(inst_vertices[i]
                                               - robots_path[-2-j][pointer])
                    if verbose:
                        print('Distance from the ' + str(j)
                              + 'th previus point:', prev_dist)
                    if (prev_dist < parameters["tolerance"]
                        and prev_dist < main_distance):
                        milestone = i
                        main_distance = prev_dist
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Pointer:',milestone)
            if milestone > -1 and milestone < len(inst_vertices):
                #------------------------CONTROL POINT------------------------
                if verbose:
                    print('Deadlock in vertex')
                if sense == 'counterclockwise':
                    if milestone == 0:
                        aux_vertex = (inst_vertices[-2]
                                      - inst_vertices[milestone])
                    else:
                        aux_vertex = (inst_vertices[milestone-1]
                                      - inst_vertices[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate =(aux_vertex*parameters["deadlock_mov"]
                                         + pos[pointer])
                elif sense == 'clockwise':
                    aux_vertex = (inst_vertices[milestone+1]
                                  - inst_vertices[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
            # If deadlock is on an edge
            else:
                #------------------------CONTROL POINT-------------------------
                if verbose:
                    print('Deadlock in edge')
                if sense == 'clockwise':
                    aux_vertex = closer["edges"][2*pointer] - pos[pointer]
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
                elif sense == 'counterclockwise':
                    aux_vertex = closer["edges"][2*pointer+1]-pos[pointer]
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
            # Make sure the point is inside BVC
            if path.Path(inst_vertices).contains_point(position_candidate):
                fixed_up_positions.append(position_candidate)
            else:
                fixed_up_positions.append(pos[pointer])
            #--------------------------CONTROL POINT---------------------------
            if verbose:
                print('-----------------------------------')
        else:
            fixed_up_positions.append(closer["point"][pointer])
    #------------------------------CONTROL POINT------------------------------
    return np.array(fixed_up_positions)

### Deadlock Test Function

`def deadlock_test(parameters, opt, bvc, pos, count, robots_path)`

Check if there are robots in deadlock situation

#### Parameters:
* **parameters&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Global parameters
* **opt&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * General options
* **bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Information of BVC
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **count&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Step
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarryas of list of floats**
    * Previous positions
    
#### Return:
* **closer_points&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Positions to avoid deadlock

In [ ]:
def deadlock_test(parameters, opt, bvc, pos, count, robots_path, closer):
    deadlock_flag = False
    deadlock_pointers = []
    if count >= parameters["previous"]:
        for i in range(parameters["robots"]):
            if not np.all(closer["point"][i] == pos["f"][i]):
                j = 0
                deadlock_local_flag = False
                distance1 = np.linalg.norm(closer["point"][i] - pos["i"][i])
                if opt["verbose"]:
                    print('Distance from', opt["palette"][i],
                          'to the closest point:', distance1)
                while (not deadlock_local_flag and j < parameters["previous"]):
                    distance2 = np.linalg.norm(closer["point"][i]
                                               - robots_path[-2-j][i])
                    if opt["verbose"]:
                        print('Distance from the '+str(j+1)+
                              'th previous point:', distance2)
                    if (distance1 < parameters["tolerance"]
                        or distance2 < parameters["tolerance"]):
                        deadlock_pointers.append(i)
                        deadlock_local_flag = True
                        deadlock_flag = True
                    j += 1
    if opt["verbose"]:
        print('Closer positions:', closer["point"])
        print('Deadlock?:', deadlock_flag,'\n')
    # BREAKING DEADLOCK
    if deadlock_flag:
        if opt["verbose"]:
            print('Cells in Deadlock:')
            for i in deadlock_pointers:
                print(opt["palette"][i], end = ' ')
            print('\nCloser_edges:', closer["edges"], '\n')
        closer["point"] = deadlock(pos["i"], bvc, parameters, robots_path,
                                   deadlock_pointers, opt["verbose"], closer)
        if opt["verbose"]:
            print('DEADLOCK ABOVE!')
            print('Previous positions:', robots_path[-2])
            print('New closer positions:', closer["point"],'\n')
    bvc.update({"in_deadlock":deadlock_pointers})
    return bvc, closer